# I. Importations des modules et des données
## 1.1 Importation des modules

In [52]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from zipfile import ZipFile
import shutil

## 1.2 Extraction des données

In [59]:
allow_extraction = False
datasets_path = r"C:\Users\axelb\Desktop\Projet DS\Datasets"
extracted_datasets_path = os.path.join(datasets_path,'Extracted')
new_dataset_filename = "Other"
new_dataset_path = os.path.join(extracted_datasets_path, new_dataset_filename)
datasets_to_regroup = ['Dataset','Painting','Schematics','Sketch','Text']

if allow_extraction:
    try:
        for dataset_filename in os.listdir(datasets_path):

            dataset_path = os.path.join(datasets_path, dataset_filename)
            dataset_zip = ZipFile(dataset_path, 'r')

            if not os.path.exists(extracted_datasets_path):
                os.makedirs(extracted_datasets_path)
                print(f"Extraction folder successfully created at '{extracted_datasets_path}'")

            print(f"Starting the extraction of '{dataset_filename}' at '{extracted_datasets_path}'")
            dataset_zip.extractall(extracted_datasets_path)
            print(f"'{dataset_filename}' was successfully extracted at '{extracted_datasets_path}'")

        dataset_zip.close()
    except:
        pass
    
    if not os.path.exists(new_dataset_path):
        os.makedirs(new_dataset_path)
        print(f"New folder successfully created at '{new_dataset_path}'")
    for dataset in os.listdir(extracted_datasets_path):
        if dataset in datasets_to_regroup:
            dataset_directory = os.path.join(extracted_datasets_path, dataset)
            print(f"Starting the copy of '{dataset_directory}' at '{new_dataset_path}'")
            shutil.copytree(dataset_directory, new_dataset_path, copy_function=shutil.move, dirs_exist_ok=True)
            print(f"'The content of {dataset_directory}' was successfully extracted at '{new_dataset_path}'")
            shutil.rmtree(dataset_directory)

Extraction folder successfully created at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
Starting the extraction of 'Dataset Livrable 1 - Painting.zip' at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
'Dataset Livrable 1 - Painting.zip' was successfully extracted at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
Starting the extraction of 'Dataset Livrable 1 - Photo.zip' at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
'Dataset Livrable 1 - Photo.zip' was successfully extracted at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
Starting the extraction of 'Dataset Livrable 1 - Schematics.zip' at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
'Dataset Livrable 1 - Schematics.zip' was successfully extracted at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
Starting the extraction of 'Dataset Livrable 1 - Sketch.zip' at 'C:\Users\axelb\Desktop\Projet DS\Datasets\Extracted'
'Dataset Livrable 1 - Sketch.zip' was successfully extracted at 'C:\

## 1.3 Importation des données

In [62]:
# Le train_set
train_set, test_set = tf.keras.preprocessing.image_dataset_from_directory(
    extracted_datasets_path,
    validation_split=0.2,
    seed=10,
    batch_size=128,
    subset = "both"
)
print(train_set.class_names)

Found 41406 files belonging to 2 classes.
Using 33125 files for training.
Using 8281 files for validation.
['Other', 'Photo']
